In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import os
from tensorflow.keras.callbacks import ModelCheckpoint

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


### Attempt 1: Decreasing Columns and Changing bins

In [3]:
# Drop the non-beneficial ID columns, 'EIN','NAME''ORGANIZATION','CLASSIFICATION'.
attempt1_df= application_df.drop(['EIN','NAME','ORGANIZATION','CLASSIFICATION'],axis=1)

In [4]:
# Determine the number of unique values in each column.
attempt1_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
USE_CASE                     5
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [5]:
# Look at APPLICATION_TYPE value counts for binning
app_type_counts = attempt1_df.APPLICATION_TYPE.value_counts()
app_type_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Determine which values to replace if counts are less than ...?
replace_application =  list(app_type_counts[app_type_counts<700].index)

# Replace in dataframe
for app in replace_application:
    attempt1_df.APPLICATION_TYPE = attempt1_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
attempt1_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
Other      804
T8         737
T7         725
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Generate our categorical variable lists
application_cat = attempt1_df.dtypes[application_df.dtypes=="object"].index.tolist()

In [8]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(attempt1_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

C:\Users\HP\anaconda3\envs\PythonData\envs\mlenv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [9]:
# Merge one-hot encoded features and drop the originals
attempt1_df = attempt1_df.merge(encode_df,left_index=True,right_index=True)
attempt1_df = attempt1_df.drop(application_cat,1)
attempt1_df.head()

C:\Users\HP\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [10]:
# Split our preprocessed data into our features and target arrays
y = attempt1_df["IS_SUCCESSFUL"].values
X = attempt1_df.drop(["IS_SUCCESSFUL"],axis=1).values
# Split the preprocessed data into a training and testing dataset
X_train,X_test, y_train, y_test = train_test_split(X,y,random_state=78)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80,input_dim=number_input_features,activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                2640      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5,101
Trainable params: 5,101
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [14]:
# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    frequency=5)

In [15]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
775/804 [===========================>..] - ETA: 0s - loss: 23797.3887 - accuracy: 0.4938
Epoch 1: saving model to checkpoints\weights.01.hdf5
804/804 [==============================] - 2s 1ms/step - loss: 23219.1680 - accuracy: 0.4959
Epoch 2/100
799/804 [============================>.] - ETA: 0s - loss: 11801.1475 - accuracy: 0.5071
Epoch 2: saving model to checkpoints\weights.02.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 11741.9834 - accuracy: 0.5068
Epoch 3/100
780/804 [============================>.] - ETA: 0s - loss: 8667.5908 - accuracy: 0.4880
Epoch 3: saving model to checkpoints\weights.03.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 8981.7295 - accuracy: 0.4891
Epoch 4/100
793/804 [============================>.] - ETA: 0s - loss: 18231.4180 - accuracy: 0.4950
Epoch 4: saving model to checkpoints\weights.04.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 18039.8633 - accuracy: 0.4955
Epoch 5/100
79

794/804 [============================>.] - ETA: 0s - loss: 0.6640 - accuracy: 0.5857
Epoch 34: saving model to checkpoints\weights.34.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.6644 - accuracy: 0.5851
Epoch 35/100
783/804 [============================>.] - ETA: 0s - loss: 0.6865 - accuracy: 0.5321
Epoch 35: saving model to checkpoints\weights.35.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.6864 - accuracy: 0.5321
Epoch 36/100
803/804 [============================>.] - ETA: 0s - loss: 0.6853 - accuracy: 0.5322
Epoch 36: saving model to checkpoints\weights.36.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.6853 - accuracy: 0.5321
Epoch 37/100
804/804 [==============================] - ETA: 0s - loss: 0.6851 - accuracy: 0.5321
Epoch 37: saving model to checkpoints\weights.37.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.6851 - accuracy: 0.5321
Epoch 38/100
790/804 [===========================

778/804 [============================>.] - ETA: 0s - loss: 0.6913 - accuracy: 0.5317
Epoch 68: saving model to checkpoints\weights.68.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.6912 - accuracy: 0.5321
Epoch 69/100
778/804 [============================>.] - ETA: 0s - loss: 0.6914 - accuracy: 0.5312
Epoch 69: saving model to checkpoints\weights.69.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.6913 - accuracy: 0.5321
Epoch 70/100
785/804 [============================>.] - ETA: 0s - loss: 0.6913 - accuracy: 0.5323
Epoch 70: saving model to checkpoints\weights.70.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.6913 - accuracy: 0.5321
Epoch 71/100
799/804 [============================>.] - ETA: 0s - loss: 0.6914 - accuracy: 0.5320
Epoch 71: saving model to checkpoints\weights.71.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.6914 - accuracy: 0.5321
Epoch 72/100
801/804 [===========================

In [16]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

#Accuracy without callbacks in fit.model statement was about 0.71

268/268 - 0s - loss: 1.1895 - accuracy: 0.7135 - 399ms/epoch - 1ms/step
Loss: 1.1895493268966675, Accuracy: 0.713469386100769


In [17]:
#Export model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")

### Attempt 2: Adding additional layers and epoch=110

In [18]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])

nn_new2 = tf.keras.models.Sequential()

# First hidden layer
nn_new2.add(tf.keras.layers.Dense(units=80,input_dim=number_input_features,activation="relu"))

# Second hidden layer
nn_new2.add(tf.keras.layers.Dense(units=30, activation="relu"))

#Third hidden layer
nn_new2.add(tf.keras.layers.Dense(units=50,activation='relu'))

# Output layer
nn_new2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_new2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 80)                2640      
                                                                 
 dense_4 (Dense)             (None, 30)                2430      
                                                                 
 dense_5 (Dense)             (None, 50)                1550      
                                                                 
 dense_6 (Dense)             (None, 1)                 51        
                                                                 
Total params: 6,671
Trainable params: 6,671
Non-trainable params: 0
_________________________________________________________________


In [19]:
# Compile the model
nn_new2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [20]:
# Train the model
fit_model = nn_new2.fit(X_train,y_train,epochs=110)

Epoch 1/110
804/804 [==============================] - 2s 2ms/step - loss: 26105.7852 - accuracy: 0.4923
Epoch 2/110
804/804 [==============================] - 1s 2ms/step - loss: 10104.3779 - accuracy: 0.5003
Epoch 3/110
804/804 [==============================] - 1s 2ms/step - loss: 2381.5020 - accuracy: 0.4885
Epoch 4/110
804/804 [==============================] - 1s 2ms/step - loss: 109.6401 - accuracy: 0.5316
Epoch 5/110
804/804 [==============================] - 1s 2ms/step - loss: 0.6991 - accuracy: 0.5321
Epoch 6/110
804/804 [==============================] - 1s 2ms/step - loss: 0.6913 - accuracy: 0.5321
Epoch 7/110
804/804 [==============================] - 1s 2ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 8/110
804/804 [==============================] - 1s 2ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 9/110
804/804 [==============================] - 1s 2ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 10/110
804/804 [==============================] - 1s 2ms/step - loss: 0.

804/804 [==============================] - 1s 2ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 82/110
804/804 [==============================] - 1s 2ms/step - loss: 0.6912 - accuracy: 0.5321
Epoch 83/110
804/804 [==============================] - 1s 2ms/step - loss: 0.6912 - accuracy: 0.5321
Epoch 84/110
804/804 [==============================] - 1s 2ms/step - loss: 0.6912 - accuracy: 0.5321
Epoch 85/110
804/804 [==============================] - 1s 2ms/step - loss: 0.6912 - accuracy: 0.5321
Epoch 86/110
804/804 [==============================] - 1s 2ms/step - loss: 0.6912 - accuracy: 0.5321
Epoch 87/110
804/804 [==============================] - 1s 2ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 88/110
804/804 [==============================] - 1s 2ms/step - loss: 0.6912 - accuracy: 0.5321
Epoch 89/110
804/804 [==============================] - 1s 2ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 90/110
804/804 [==============================] - 1s 2ms/step - loss: 0.6912 - accuracy: 

In [21]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_new2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6910 - accuracy: 0.5445 - 455ms/epoch - 2ms/step
Loss: 0.6909942626953125, Accuracy: 0.5444898009300232


### Attempt 3: Adding hidden layers with different activation functions, increasing neurons and epoch=80

In [22]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])

nn_new1 = tf.keras.models.Sequential()

# First hidden layer
nn_new1.add(tf.keras.layers.Dense(units=100,input_dim=number_input_features,activation="relu"))

# Second hidden layer
nn_new1.add(tf.keras.layers.Dense(units=80, activation="relu"))

#Third hidden layer
nn_new1.add(tf.keras.layers.Dense(units=50,activation='relu'))

#Fourth hidden layer
nn_new1.add(tf.keras.layers.Dense(units=30,activation='sigmoid'))

# Output layer
nn_new1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_new1.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 100)               3300      
                                                                 
 dense_8 (Dense)             (None, 80)                8080      
                                                                 
 dense_9 (Dense)             (None, 50)                4050      
                                                                 
 dense_10 (Dense)            (None, 30)                1530      
                                                                 
 dense_11 (Dense)            (None, 1)                 31        
                                                                 
Total params: 16,991
Trainable params: 16,991
Non-trainable params: 0
_________________________________________________________________


In [23]:
# Compile the model
nn_new1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [24]:
# Train the model
fit_model = nn_new1.fit(X_train,y_train,epochs=80)

Epoch 1/80
804/804 [==============================] - 2s 2ms/step - loss: 0.7012 - accuracy: 0.5233
Epoch 2/80
804/804 [==============================] - 1s 2ms/step - loss: 0.6914 - accuracy: 0.5321
Epoch 3/80
804/804 [==============================] - 1s 2ms/step - loss: 0.6913 - accuracy: 0.5311
Epoch 4/80
804/804 [==============================] - 1s 2ms/step - loss: 0.6915 - accuracy: 0.5321
Epoch 5/80
804/804 [==============================] - 1s 2ms/step - loss: 0.6914 - accuracy: 0.5321
Epoch 6/80
804/804 [==============================] - 1s 2ms/step - loss: 0.6915 - accuracy: 0.5321
Epoch 7/80
804/804 [==============================] - 1s 2ms/step - loss: 0.6915 - accuracy: 0.5321
Epoch 8/80
804/804 [==============================] - 1s 2ms/step - loss: 0.6915 - accuracy: 0.5321
Epoch 9/80
804/804 [==============================] - 1s 2ms/step - loss: 0.6915 - accuracy: 0.5311
Epoch 10/80
804/804 [==============================] - 1s 2ms/step - loss: 0.6912 - accuracy: 0.5321

In [25]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_new1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.7267 - accuracy: 0.4675 - 432ms/epoch - 2ms/step
Loss: 0.726708710193634, Accuracy: 0.46752187609672546
